https://m.ensembl.org/info/docs/api/core/core_schema.html#associated_group

gene: all
xref: display_label (gene name) and description

In [193]:
%load_ext blackcellmagic

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic


In [8]:
db = "taeniopygia_guttata_core_105_12"
searchword = "GABA"
limit = 20

____

In [9]:
import pandas as pd
import mysql.connector as sql

db_connection = sql.connect(host='ensembldb.ensembl.org', 
                            database=db, 
                            user='anonymous', 
                            password='')

In [10]:
query = f"""
SELECT gene.stable_id, gene_attrib.value, gene.description, xref.description, gene.biotype
FROM gene
LEFT JOIN xref ON gene.display_xref_id = xref.xref_id
LEFT JOIN gene_attrib ON gene.gene_id = gene_attrib.gene_id
WHERE (gene_attrib.attrib_type_id = 4) 
AND (gene.description LIKE '%{searchword}%' OR xref.description LIKE '%{searchword}%')
"""

In [41]:
query = f"""
SELECT *
FROM gene
LIMIT 10
"""

In [42]:
df = pd.read_sql(query, con=db_connection)
# Order using pandas instead of SQL to increase speed
# df = df.sort_values("gene_id").reset_index(drop=True)
df

,gene_id,biotype,analysis_id,seq_region_id,seq_region_start,seq_region_end,seq_region_strand,display_xref_id,source,description,is_current,canonical_transcript_id,stable_id,version,created_date,modified_date
0,1,protein_coding,9,1,34955,72180,1,661353.0,ensembl,"discoidin, CUB and LCCL domain containing 2 [S...",1,1,ENSTGUG00000013637,2,2009-01-14 12:02:56,2019-09-24 14:34:38
1,2,protein_coding,9,1,45219,302771,-1,655188.0,ensembl,cms1 ribosomal small subunit homolog [Source:N...,1,4,ENSTGUG00000013635,2,2009-01-14 12:02:56,2019-09-24 14:34:38
2,3,protein_coding,9,1,74112,115790,-1,NaN,ensembl,None,1,6,ENSTGUG00000020928,1,2019-09-24 14:34:38,2019-09-24 14:34:38
3,4,protein_coding,9,1,115954,334268,1,687272.0,ensembl,filamin A interacting protein 1 like [Source:N...,1,8,ENSTGUG00000013634,2,2009-01-14 12:02:56,2019-09-24 14:34:38
4,5,protein_coding,9,1,159330,160474,-1,NaN,ensembl,None,1,10,ENSTGUG00000027178,1,2019-09-24 14:34:38,2019-09-24 14:34:38
5,6,protein_coding,9,1,303920,334270,1,661671.0,ensembl,TBC1 domain family member 23 [Source:HGNC Symb...,1,11,ENSTGUG00000013633,2,2009-01-14 12:02:56,2019-09-24 14:34:38
6,7,protein_coding,9,1,331197,351031,1,663909.0,ensembl,nitrilase family member 2 [Source:HGNC Symbol;...,1,14,ENSTGUG00000013632,2,2009-01-14 12:02:56,2019-09-24 14:34:38
7,8,protein_coding,9,1,349213,372968,-1,665173.0,ensembl,translocase of outer mitochondrial membrane 70...,1,16,ENSTGUG00000013631,2,2009-01-14 12:02:56,2019-09-24 14:34:38
8,9,protein_coding,9,1,375679,376157,1,NaN,ensembl,None,1,17,ENSTGUG00000013630,2,2009-01-14 12:02:56,2019-09-24 14:34:38
9,10,protein_coding,9,1,391784,417140,1,668276.0,ensembl,transmembrane protein 45A [Source:NCBI gene;Ac...,1,18,ENSTGUG00000013629,2,2009-01-14 12:02:56,2019-09-24 14:34:38


Notes:  
external_synonym - gene names?  
xref.display_label or dbprimary_acc sometimes have ENS numbers (but this is not the case for "GABA" searchresults)  
exon.stable_id - ENS numbers  
gene_attrib.value WHERE attrib_type_id = 4 - gene name  (only for human and mouse?)  
protein_feature.hit_description - includes "GABA" hits  
gene_archive - includes matched Gene, transcript and translation IDs 

___

In [22]:
import pandas as pd
# pure-Python MySQL client library
import pymysql

db = "taeniopygia_guttata_core_105_12"
searchword = "GABA"
limit = 20

database = pymysql.connect(
host="ensembldb.ensembl.org",
user="anonymous",
password="",
database=db
)

In [23]:
cursor = database.cursor()
# Union between left and right join replaces full outer join in this vesion of SQL
cursor.execute(
    f"""
    SELECT *
    FROM gene 
    WHERE description LIKE '%{searchword}%';
    """
)

results = cursor.fetchall()

# Get gene table column names
cursor.execute(f"show columns from gene;")
header_tuples = cursor.fetchall()
header1 = [a_tuple[0] for a_tuple in header_tuples]

# Save search results from gene table in data frame
df_temp = pd.DataFrame(
    results,
    columns = header1,
)

In [24]:
len(df_temp)

6

In [244]:
cursor = database.cursor()

# ## Select genes based on search word in "description" column of "gene" table
# cursor.execute(f"select * from gene where description like '%{searchword}%';")
# res1 = cursor.fetchall()

# # Get gene table column names
# cursor.execute(f"show columns from gene;")
# header_tuples = cursor.fetchall()
# header1 = [a_tuple[0] for a_tuple in header_tuples]

# # Save search results from gene table in data frame
# df_temp1 = pd.DataFrame(
#     res1,
#     columns=header1,
# )

# # "display_xref_id" links to "xref_id" in "xref" table
# display_xref_ids = df_temp["display_xref_id"].values

# # Find search results form gene table in xref table
# for xref_id in display_xref_ids:
#     cursor.execute(f"select external_db_id, description from xref where xref_id like '%{xref_id}%';")
#     res2 = cursor.fetchall()
#     header2 = ["ext database ID", "ext description"]


cursor.execute(f"select description from gene where description like '%{searchword}%';")
gene_description = cursor.fetchall()


## Select genes based on search word in "description" column of "xref" table
# This table contains external web-data displayed on the Ensembl website
cursor.execute(f"select description from xref where description like '%{searchword}%';")
ext_description = cursor.fetchall()
# header2 = ["ext database ID", "ext description"]

cursor.close()
# # external_db_ids = 
# ext_dbs = []
# for ext_id in external_db_ids:
#     ## Get database name of xref genes from "db_name" column of "external_db" table
#     cursor.execute(f"select db_name from external_db where external_db_id like '%{external_db_ids}%';")
#     ext_dbs.append

## Get gene names from "value" column in gene_attrib table
cursor.execute(f"select value from gene_attrib limit {limit};")
gene_names = cursor.fetchall()

header3 = ["gene_name"]

In [249]:
len(gene_description)

16

In [248]:
len(ext_description)

350

In [21]:
cursor = database.cursor()
# Union between left and right join replaces full outer join in this vesion of SQL
cursor.execute(
    f"""
    SELECT gene.*, xref.description, gene_attrib.value 
    FROM gene 
    LEFT JOIN xref ON gene.display_xref_id=xref.xref_id 
    WHERE gene.description LIKE '%{searchword}%' OR xref.description LIKE '%{searchword}%'
    UNION
    SELECT gene.*, xref.description, gene_attrib.value 
    FROM gene   
    RIGHT JOIN gene_attrib ON gene.gene_id=gene_attrib.gene_id 
    WHERE gene.description LIKE '%{searchword}%' OR xref.description LIKE '%{searchword}%';
    """
)

results = cursor.fetchall()
cursor.close()

# Get gene table column names
cursor.execute(f"show columns from gene;")
header_tuples = cursor.fetchall()
header1 = [a_tuple[0] for a_tuple in header_tuples]
# Define column name for xref description and gene_attrib.value
header2 = ["xref description", "gene name"]

# Save search results from gene table in data frame
df_temp = pd.DataFrame(
    results,
    columns = header1 + header2,
)

OperationalError: (1054, "Unknown column 'gene_attrib.value' in 'field list'")

In [15]:
cursor = database.cursor()
# Union between left and right join replaces full outer join in this vesion of SQL
cursor.execute(
    f"""
    SELECT gene.*, xref.description, gene_attrib.value 
    FROM gene 
    LEFT JOIN xref ON gene.display_xref_id=xref.xref_id 
    INNER JOIN gene_attrib ON gene.gene_id=gene_attrib.gene_id 
    WHERE gene_attrib.attrib_type_id = 4 AND gene.description LIKE '%{searchword}%' OR xref.description LIKE '%{searchword}%'
    UNION
    SELECT gene.*, xref.description, gene_attrib.value 
    FROM gene 
    RIGHT JOIN xref ON gene.display_xref_id=xref.xref_id 
    INNER JOIN gene_attrib ON gene.gene_id=gene_attrib.gene_id 
    WHERE gene_attrib.attrib_type_id = 4 AND gene.description LIKE '%{searchword}%' OR xref.description LIKE '%{searchword}%';
    """
)

results = cursor.fetchall()
cursor.close()

# Get gene table column names
cursor.execute(f"show columns from gene;")
header_tuples = cursor.fetchall()
header1 = [a_tuple[0] for a_tuple in header_tuples]
# Define column name for xref description and gene_attrib.value
header2 = ["xref description", "gene name"]

# Save search results from gene table in data frame
df_temp = pd.DataFrame(
    results,
    columns = header1 + header2,
)

In [16]:
df_temp

,gene_id,biotype,analysis_id,seq_region_id,seq_region_start,seq_region_end,seq_region_strand,display_xref_id,source,description,is_current,canonical_transcript_id,stable_id,version,created_date,modified_date,xref description,gene name
0,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578.0,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37,GABARAPL1 antisense RNA 1,RP11-656E20.5
1,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578.0,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37,GABARAPL1 antisense RNA 1,38.10
2,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578.0,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37,GABARAPL1 antisense RNA 1,OTTHUMG00000168529
3,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578.0,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37,GABARAPL1 antisense RNA 1,processed_transcript
4,18340,protein_coding,8353,131546,10212458,10223128,1,6355576.0,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 1,GABARAPL1
5,18340,protein_coding,8353,131546,10212458,10223128,1,6355576.0,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 1,44.11
6,18340,protein_coding,8353,131546,10212458,10223128,1,6355576.0,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 1,OTTHUMG00000168411
7,18340,protein_coding,8353,131546,10212458,10223128,1,6355576.0,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 1,ENST00000266458
8,20765,processed_pseudogene,8354,131546,32072734,32072880,1,NaN,havana,"gamma-aminobutyric acid (GABA) A receptor, pi ...",1,63959,ENSG00000257852,1,2011-05-11 16:49:31,2011-05-11 16:49:31,None,RP11-843B15.1
9,21715,processed_pseudogene,8354,131560,14690382,14690619,1,NaN,havana,diazepam binding inhibitor (GABA receptor modu...,1,66907,ENSG00000255365,1,2010-11-01 15:31:55,2010-11-01 15:31:55,None,RP11-776K3.1


In [ ]:
sql1 = f"""
SELECT gene.*, gene_attrib.value, xref.description
FROM gene 
INNER JOIN gene_attrib ON gene.gene_id=gene_attrib.gene_id 
LEFT JOIN xref ON gene.display_xref_id=xref.xref_id
WHERE gene_attrib.attrib_type_id = 4 AND gene.description LIKE '%{searchword}%'
"""

# sql2 = f"""
# SELECT gene.*, gene_attrib.value, xref.description
# FROM gene
# INNER JOIN gene_attrib ON gene.gene_id=gene_attrib.gene_id
# LEFT JOIN xref ON gene.display_xref_id=xref.xref_id
# WHERE gene_attrib.attrib_type_id = 4 AND xref.description LIKE '%{searchword}%';
# """

sql2 = f"""
SELECT gene.*, xref.description
FROM gene
LEFT JOIN xref ON gene.display_xref_id=xref.xref_id
WHERE xref.description LIKE '%{searchword}%';
"""

In [69]:
cursor = database.cursor()
# Union between left and right join replaces full outer join in this vesion of SQL
cursor.execute(
sql2
)

results = cursor.fetchall()
cursor.close()

# Get gene table column names
cursor.execute(f"show columns from gene;")
header_tuples = cursor.fetchall()
header1 = [a_tuple[0] for a_tuple in header_tuples]
# Define column name for xref description and gene_attrib.value
header2 = ["xref_description"]

# Save search results from gene table in data frame
df_temp = pd.DataFrame(
    results,
    columns = header1 + header2,
)

In [70]:
df_temp

,gene_id,biotype,analysis_id,seq_region_id,seq_region_start,seq_region_end,seq_region_strand,display_xref_id,source,description,is_current,canonical_transcript_id,stable_id,version,created_date,modified_date,xref_description
0,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37,GABARAPL1 antisense RNA 1
1,18340,protein_coding,8353,131546,10212458,10223128,1,6355576,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 1
2,44845,protein_coding,8353,131549,75566375,75577881,1,6355579,ensembl_havana,GABA type A receptor associated protein like 2...,1,149153,ENSG00000034713,8,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor associated protein like 2
3,53729,TEC,8354,131558,90347587,90349437,1,4017258,havana,GABA type A receptor associated protein like 3...,1,187564,ENSG00000279980,1,2014-08-13 17:29:05,2014-08-13 17:29:05,GABA type A receptor associated protein like 3...
4,53730,processed_pseudogene,8354,131558,90348844,90349197,-1,6355582,havana,GABA type A receptor associated protein like 3...,1,187568,ENSG00000238244,3,2009-05-19 09:47:17,2014-06-09 10:49:07,GABA type A receptor associated protein like 3...
5,60597,protein_coding,8353,131537,44954585,44993341,1,6345391,ensembl_havana,CLPTM1 regulator of GABA type A receptor forwa...,1,223716,ENSG00000104853,16,2008-04-29 11:17:41,2018-11-21 17:23:49,CLPTM1 regulator of GABA type A receptor forwa...
6,62468,protein_coding,8353,131554,7240008,7242449,-1,6355574,ensembl_havana,GABA type A receptor-associated protein [Sourc...,1,233989,ENSG00000170296,10,2008-04-29 11:17:41,2018-11-21 17:23:49,GABA type A receptor-associated protein


In [77]:
table = "xref"

cursor = database.cursor()
cursor.execute(
    f"""
    SELECT *
    FROM {table}
    WHERE xref.description LIKE '%SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRANSPORTER, GABA)%';
    """
)

results = cursor.fetchall()
cursor.close()

# Get table column names
cursor.execute(f"SHOW COLUMNS FROM {table};")
header_tuples = cursor.fetchall()
header = [a_tuple[0] for a_tuple in header_tuples]

# Save search results in data frame
df_temp = pd.DataFrame(
    results,
    columns = header,
)

df_temp

,xref_id,external_db_id,dbprimary_acc,display_label,version,description,info_type,info_text
0,6122814,1510,137165,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,0,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,DEPENDENT,
1,6143999,1510,615097,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,0,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,DEPENDENT,
2,6136633,1510,607952,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,0,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,DEPENDENT,


In [86]:
cursor = database.cursor()
cursor.execute(
    f"""
    SELECT gene.*, xref.*
    FROM gene
    RIGHT JOIN xref ON gene.display_xref_id=xref.xref_id
    WHERE xref.xref_id = 6122814 OR xref.xref_id = 6143999 OR xref.xref_id = 6136633;
    """
)

results = cursor.fetchall()
cursor.close()

# Get table column names
cursor.execute(f"SHOW COLUMNS FROM gene;")
header_tuples = cursor.fetchall()
header1 = [a_tuple[0] for a_tuple in header_tuples]

cursor.execute(f"SHOW COLUMNS FROM xref;")
header_tuples = cursor.fetchall()
header2 = [a_tuple[0] for a_tuple in header_tuples]

# Save search results in data frame
df_temp = pd.DataFrame(
    results,
    columns = header1 + header2,
)

df_temp

,gene_id,biotype,analysis_id,seq_region_id,seq_region_start,seq_region_end,seq_region_strand,display_xref_id,source,description,...,created_date,modified_date,xref_id,external_db_id,dbprimary_acc,display_label,version,description,info_type,info_text
0,None,None,None,None,None,None,None,None,None,None,...,None,None,6122814,1510,137165,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,0,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,DEPENDENT,
1,None,None,None,None,None,None,None,None,None,None,...,None,None,6136633,1510,607952,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,0,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,DEPENDENT,
2,None,None,None,None,None,None,None,None,None,None,...,None,None,6143999,1510,615097,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,0,SOLUTE CARRIER FAMILY 6 (NEUROTRANSMITTER TRAN...,DEPENDENT,


In [7]:
cursor = database.cursor()
cursor.execute(
    f"""
    SELECT *
    FROM gene
    WHERE display_xref_id IN
    (SELECT xref_id
    FROM xref
    WHERE description LIKE '%{searchword}%');
    """
)

results = cursor.fetchall()
cursor.close()

# Get table column names
cursor.execute(f"SHOW COLUMNS FROM gene;")
header_tuples = cursor.fetchall()
header1 = [a_tuple[0] for a_tuple in header_tuples]

# Save search results in data frame
df_temp = pd.DataFrame(
    results,
    columns = header1,
)

df_temp

,gene_id,biotype,analysis_id,seq_region_id,seq_region_start,seq_region_end,seq_region_strand,display_xref_id,source,description,is_current,canonical_transcript_id,stable_id,version,created_date,modified_date
0,17657,lncRNA,8354,131546,10214161,10214761,-1,6355578,havana,GABARAPL1 antisense RNA 1 [Source:HGNC Symbol;...,1,53895,ENSG00000255958,1,2011-02-14 13:38:37,2011-02-14 13:38:37
1,18340,protein_coding,8353,131546,10212458,10223128,1,6355576,ensembl_havana,GABA type A receptor associated protein like 1...,1,55864,ENSG00000139112,11,2008-04-29 11:17:41,2018-11-21 17:23:49
2,44845,protein_coding,8353,131549,75566375,75577881,1,6355579,ensembl_havana,GABA type A receptor associated protein like 2...,1,149153,ENSG00000034713,8,2008-04-29 11:17:41,2018-11-21 17:23:49
3,53729,TEC,8354,131558,90347587,90349437,1,4017258,havana,GABA type A receptor associated protein like 3...,1,187564,ENSG00000279980,1,2014-08-13 17:29:05,2014-08-13 17:29:05
4,53730,processed_pseudogene,8354,131558,90348844,90349197,-1,6355582,havana,GABA type A receptor associated protein like 3...,1,187568,ENSG00000238244,3,2009-05-19 09:47:17,2014-06-09 10:49:07
5,60597,protein_coding,8353,131537,44954585,44993341,1,6345391,ensembl_havana,CLPTM1 regulator of GABA type A receptor forwa...,1,223716,ENSG00000104853,16,2008-04-29 11:17:41,2018-11-21 17:23:49
6,62468,protein_coding,8353,131554,7240008,7242449,-1,6355574,ensembl_havana,GABA type A receptor-associated protein [Sourc...,1,233989,ENSG00000170296,10,2008-04-29 11:17:41,2018-11-21 17:23:49


Only 7 of the ~170 genes with "GABA" in their xref.description have linked IDs in the gene tables...

In [26]:
cursor = database.cursor()
cursor.execute(
    f"""
    SELECT *
    FROM gene
    WHERE description LIKE '%{searchword}%';
    """
)

results = cursor.fetchall()

cursor.close()

# # Get table column names
# cursor.execute(f"SHOW COLUMNS FROM gene;")
# header_tuples = cursor.fetchall()
# header1 = [a_tuple[0] for a_tuple in header_tuples]

# # Get table column names
# cursor.execute(f"SHOW COLUMNS FROM xref;")
# header_tuples = cursor.fetchall()
# header2 = [a_tuple[0] for a_tuple in header_tuples]

# # Save search results in data frame
# df_temp = pd.DataFrame(
#     results,
#     columns = header1 + header2,
# )

pd.read_sql(results, "SQLAlchemy")

ImportError: Using URI string without sqlalchemy installed.